### Q1

In [1]:
import cv2
import numpy as np

# Function to calculate SSD (Sum of Squared Differences)
def ssd(image1, image2):
    diff = image1 - image2
    return np.sum(diff ** 2)

# Function to find the best match for the object in the scene
def find_best_match(cropped_region, dataset):
    best_match_index = -1
    best_match_score = float('inf')
    for i, image in enumerate(dataset):
        score = ssd(cropped_region, image)
        if score < best_match_score:
            best_match_score = score
            best_match_index = i
    return best_match_index, best_match_score

# Load your video
video_path = 'cv video for assi.mp4'
cap = cv2.VideoCapture(video_path)

# Capture a frame from the video
fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
duration = frame_count // fps

# Extract frames
dataset = []
while True:
    ret, frame = cap.read()
    if not ret:
        break
    dataset.append(frame)
    cv2.imshow('Frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Select a frame for ROI selection
selected_frame = dataset[75]

# Select ROI using cv2.selectROI()
roi = cv2.selectROI("Frame", selected_frame)

# Crop the region of interest from the selected frame and resize it to match the dataset
roi_x, roi_y, roi_width, roi_height = roi
cropped_region = cv2.resize(selected_frame[roi_y:roi_y+roi_height, roi_x:roi_x+roi_width], (selected_frame.shape[1], selected_frame.shape[0]))

# Find the best match for the cropped region in the dataset
best_match_index, best_match_score = find_best_match(cropped_region, dataset)

# Save the selected frame, ROI, and best match
cv2.imwrite('selected_frame.jpg', selected_frame)
cv2.imwrite('roi.jpg', cropped_region)
cv2.imwrite('best_match.jpg', dataset[best_match_index])

# Display the best match
cv2.imshow('Cropped Region', cropped_region)
cv2.imshow('Best Match', dataset[best_match_index])
cv2.waitKey(0)

# Release the video capture and close windows
cap.release()
cv2.destroyAllWindows()

##Q3


In [21]:
import cv2
import numpy as np

image1 = cv2.imread("marker1.png")
image2 = cv2.imread("marker2.png")
Z = 300


x, y, w, h = [100,200,50,1]
cv2.rectangle(image1, (x, y), (x + w, y + h), (0, 255, 0), 5)

Ip1x = (x + (x+w))//2
Ip1Y = (y + (y+h))
'''mask_red_2 = cv2.inRange(image2, lower_red, upper_red)
contours, _ = cv2.findContours(mask_red_2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)'''

'''for c in contours:
    rect = cv2.boundingRect(c)
    if rect[2] < 1 or rect[3] < 1:
        continue'''
x, y, w, h = [200,300,100,1]
cv2.rectangle(image2, (x, y), (x + w, y + h), (0, 255, 0), 5)

Ip2x = (x + (x+w))//2
Ip2Y = (y + (y+h))//2

dist = 150

focal = 24

D = round((dist * focal)/abs(Ip1x - Ip2x), 2)

print("Distance: {} mm".format(D))
cv2.waitKey(0)

Distance: 28.8 mm


-1

##Q5


In [7]:
import cv2
import numpy as np
import os

# Function to calculate SSD (Sum of Squared Differences)
def ssd(image1, image2):
    diff = image1 - image2
    return np.sum(diff ** 2)

# Create a directory to save results
save_dir = 'results'
os.makedirs(save_dir, exist_ok=True)

# Load the video
video_path = 'cv video for assi.mp4'  # Replace 'your_video.mp4' with the path to your video
cap = cv2.VideoCapture(video_path)

# Capture a frame from the video
fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
duration = frame_count // fps

# Extract frames from the video
dataset = []
while True:
    ret, frame = cap.read()
    if not ret:
        break
    dataset.append(frame)

# Select a frame from the dataset
selected_frame = dataset[75]

# Select a region of interest (ROI) from the selected frame
roi = cv2.selectROI("Select ROI", selected_frame)
cv2.destroyAllWindows()

# Crop the ROI from the selected frame
roi_x, roi_y, roi_width, roi_height = roi
reference_roi = selected_frame[int(roi_y):int(roi_y + roi_height), int(roi_x):int(roi_x + roi_width)]

# Save the selected ROI
cv2.imwrite(os.path.join(save_dir, 'selected_roi.jpg'), reference_roi)

# Initialize lists to store matched frames and their corresponding SSD scores
matched_frames = []
ssd_scores = []

# Compare the reference ROI with randomly picked frames from the dataset
for i in range(10):
    # Pick a random frame from the dataset
    random_frame_index = np.random.randint(0, len(dataset))
    random_frame = dataset[random_frame_index]
    
    # Crop ROI from the random frame
    random_roi = random_frame[int(roi_y):int(roi_y + roi_height), int(roi_x):int(roi_x + roi_width)]
    
    # Calculate SSD score between reference ROI and random ROI
    score = ssd(reference_roi, random_roi)
    
    # Store the matched frame and its SSD score
    matched_frames.append(random_frame)
    ssd_scores.append(score)

    # Write the SSD score on the matched frame
    cv2.putText(random_frame, f"SSD Score: {score}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Save the matched frame
    cv2.imwrite(os.path.join(save_dir, f'matched_frame_{i+1}ssd{score}.jpg'), random_frame)

# Display the selected ROI
cv2.imshow("Selected ROI", reference_roi)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Display the matched frames with their SSD scores
for i in range(10):
    cv2.imshow(f"Matched Frame {i+1} - SSD Score: {ssd_scores[i]}", matched_frames[i])
    cv2.waitKey(0)

# Release the video capture
cap.release()
cv2.destroyAllWindows()